In [5]:
import cv2
import os
import numpy as np
import pandas as pd

src_path = r"F:\Work\SOCAAutomation\Dataset\Converted"
dst_path = r"F:\Work\SOCAAutomation\Dataset\Images\SplitVideos"
excel = pd.read_excel(r"F:\Work\SOCAAutomation\Dataset\DatasetAnalysisReport.xlsx", sheet_name = 'Video Extraction')

output_folder = "Frames"
os.makedirs(output_folder, exist_ok=True)

In [6]:
excel_group = pd.DataFrame()
excel_group['Start Time'] = excel.groupby(['Folder', 'Video File'])['Start Time'].apply(list)
excel_group['End Time'] = excel.groupby(['Folder', 'Video File'])['End Time'].apply(list)
excel_group.reset_index(inplace = True)
excel_group.head()

,Folder,Video File,Start Time,End Time
0,04_07_2024,AXIS 213 - 10.1.5.173 2024-07-04_11_51_08_772,"[02;51, 02;56, 03;12, 03;19, 03;35, 03;42, 03;...","[02;53, 02;59, 03;14, 03;21, 03;37, 03;43, 03;..."
1,09_07_2024,AXIS 213 - 10.1.5.173 2024-07-09_11_30_00_000,"[02;16, 02;19, 02;23, 03;00, 03;04, 03;26, 03;...","[02;18, 02;21, 02;25, 03;03, 03;06, 03;28, 03;..."
2,09_07_2024,AXIS 213 - 10.1.5.173 2024-07-09_14_44_00_000,"[00;24, 00;41]","[00;31, 00;43]"
3,14_08_2024,AXIS 213 - 10.1.5.173 2024-08-14_15_34_48_759,"[08;30, 13;12, 13;22, 13;26, 13;52]","[09;56, 13;14, 13;24, 13;50, 14;18]"
4,22_08_2024,AXIS 213 - 10.1.5.173 2024-08-22_10_40_00_000,"[04;48, 04;53, 05;05, 05;12, 05;15, 06;19, 06;...","[04;51, 04;54, 05;05, 05;14, 05;17, 06;21, 06;..."


In [7]:
excel_group

,Folder,Video File,Start Time,End Time
0,04_07_2024,AXIS 213 - 10.1.5.173 2024-07-04_11_51_08_772,"[02;51, 02;56, 03;12, 03;19, 03;35, 03;42, 03;...","[02;53, 02;59, 03;14, 03;21, 03;37, 03;43, 03;..."
1,09_07_2024,AXIS 213 - 10.1.5.173 2024-07-09_11_30_00_000,"[02;16, 02;19, 02;23, 03;00, 03;04, 03;26, 03;...","[02;18, 02;21, 02;25, 03;03, 03;06, 03;28, 03;..."
2,09_07_2024,AXIS 213 - 10.1.5.173 2024-07-09_14_44_00_000,"[00;24, 00;41]","[00;31, 00;43]"
3,14_08_2024,AXIS 213 - 10.1.5.173 2024-08-14_15_34_48_759,"[08;30, 13;12, 13;22, 13;26, 13;52]","[09;56, 13;14, 13;24, 13;50, 14;18]"
4,22_08_2024,AXIS 213 - 10.1.5.173 2024-08-22_10_40_00_000,"[04;48, 04;53, 05;05, 05;12, 05;15, 06;19, 06;...","[04;51, 04;54, 05;05, 05;14, 05;17, 06;21, 06;..."
5,22_08_2024,AXIS 213 - 10.1.5.173 2024-08-22_13_40_00_000,"[3;26, 3;29, 3;30, 3;32, 3;35, 3;48, 4;08, 4;1...","[3;28, 3;31, 3;32, 3;34, 3;37, 3;50, 4;15, 4;1..."
6,22_08_2024,AXIS 213 - 10.1.5.173 2024-08-22_15_45_00_000,"[9;08, 9;12, 9;15, 9;23]","[9;10, 9;14, 9;17, 9;24]"


In [11]:
fps = 30
for i in range(len(excel_group)):
    file_path = src_path + "/" + excel_group.loc[i, 'Folder'] + "/" + excel_group.loc[i, 'Video File'] + ".mp4"
    folder = dst_path + "/" + excel_group.loc[i, 'Video File'].split()[-1]
#     if folder == dst_path + "/" + '2024-08-22_10_40_00_000':
#         print(f"Folder {folder} is skipped")
#         continue
    cap = cv2.VideoCapture(file_path)
    segment = 1
    start_time_list = list(excel_group.loc[i, 'Start Time'])
    end_time_list = list(excel_group.loc[i, 'End Time'])
    for j in range(len(start_time_list)):
        
        start_time = start_time_list[j]
        end_time = end_time_list[j]
        
        print(f"Starting Segment {segment} => {start_time} to {end_time}")
        output_folder = folder +  "/" + str(segment)
        os.makedirs(output_folder, exist_ok=True)
    
        start_min = int(start_time.split(";")[0])
        start_sec = int(start_time.split(";")[1])
        start_sec = start_min * 60 + start_sec

        end_min = int(end_time.split(";")[0])
        end_sec = int(end_time.split(";")[1]) + 1
        end_sec = end_min * 60 + end_sec

        start_frame = int(start_sec * fps)
        end_frame   = int(end_sec * fps)

        frame_no = start_frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        print(f"Start Frame: {start_frame}, End Frame: {end_frame}")
        while frame_no <= end_frame:
            ret, frame = cap.read()
            if not ret:
                break

            filename = os.path.join(output_folder, f"frame_{frame_no:06d}.jpg")
            cv2.imwrite(filename, frame)

            frame_no += 1
        segment += 1

    cap.release()


Starting Segment 1 => 02;51 to 02;53
Start Frame: 5130, End Frame: 5220
Starting Segment 2 => 02;56 to 02;59
Start Frame: 5280, End Frame: 5400
Starting Segment 3 => 03;12 to 03;14
Start Frame: 5760, End Frame: 5850
Starting Segment 4 => 03;19 to 03;21
Start Frame: 5970, End Frame: 6060
Starting Segment 5 => 03;35 to 03;37
Start Frame: 6450, End Frame: 6540
Starting Segment 6 => 03;42 to 03;43
Start Frame: 6660, End Frame: 6720
Starting Segment 7 => 03;53 to 03;55
Start Frame: 6990, End Frame: 7080
Starting Segment 8 => 04;14 to 04;16
Start Frame: 7620, End Frame: 7710
Starting Segment 9 => 05;01 to 05;03
Start Frame: 9030, End Frame: 9120
Starting Segment 10 => 07;30 to 07;32
Start Frame: 13500, End Frame: 13590
Starting Segment 1 => 02;16 to 02;18
Start Frame: 4080, End Frame: 4170
Starting Segment 2 => 02;19 to 02;21
Start Frame: 4170, End Frame: 4260
Starting Segment 3 => 02;23 to 02;25
Start Frame: 4290, End Frame: 4380
Starting Segment 4 => 03;00 to 03;03
Start Frame: 5400, End F

In [10]:
list(excel_group.loc[i, 'Start Time'])

['02;51',
 '02;56',
 '03;12',
 '03;19',
 '03;35',
 '03;42',
 '03;53',
 '04;14',
 '05;01',
 '07;30']

# Prepared by Muthu